In [1]:
import os
import pandas as pd

In [9]:
ruta = 'Datos/'
df_contamination = pd.DataFrame()

for file in os.listdir(ruta):
    df_file = pd.read_csv(ruta + file, sep=';')
    df_contamination = pd.concat([df_contamination, df_file], ignore_index = True)

df_contamination.head()


,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
0,28,79,4,1,28079004_1_38,2018,4,1,2.0,V,...,2.0,V,2.0,V,3.0,V,4.0,V,3.0,V
1,28,79,4,1,28079004_1_38,2018,4,2,2.0,V,...,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V
2,28,79,4,1,28079004_1_38,2018,4,3,2.0,V,...,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V
3,28,79,4,1,28079004_1_38,2018,4,4,2.0,V,...,2.0,V,2.0,V,2.0,V,2.0,V,2.0,V
4,28,79,4,1,28079004_1_38,2018,4,5,2.0,V,...,2.0,V,2.0,V,4.0,V,5.0,V,4.0,V


In [12]:
df_contamination = df_contamination[df_contamination['MAGNITUD'] == 8]

,PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H01,V01,...,H20,V20,H21,V21,H22,V22,H23,V23,H24,V24
90,28,79,4,8,28079004_8_8,2018,4,1,21.0,V,...,46.0,V,57.0,V,82.0,V,85.0,V,77.0,V
91,28,79,4,8,28079004_8_8,2018,4,2,67.0,V,...,22.0,V,26.0,V,38.0,V,15.0,V,15.0,V
92,28,79,4,8,28079004_8_8,2018,4,3,14.0,V,...,23.0,V,27.0,V,30.0,V,23.0,V,20.0,V
93,28,79,4,8,28079004_8_8,2018,4,4,8.0,V,...,14.0,V,19.0,V,22.0,V,28.0,V,23.0,V
94,28,79,4,8,28079004_8_8,2018,4,5,20.0,V,...,45.0,V,68.0,V,99.0,V,93.0,V,84.0,V
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54733,28,79,60,8,28079060_8_8,2018,12,27,79.0,V,...,75.0,V,90.0,V,86.0,V,67.0,V,48.0,V
54734,28,79,60,8,28079060_8_8,2018,12,28,58.0,V,...,38.0,V,38.0,V,35.0,V,36.0,V,18.0,V
54735,28,79,60,8,28079060_8_8,2018,12,29,30.0,V,...,67.0,V,59.0,V,56.0,V,42.0,V,16.0,V
54736,28,79,60,8,28079060_8_8,2018,12,30,10.0,V,...,94.0,V,92.0,V,74.0,V,87.0,V,74.0,V
